In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('PySparkMl').getOrCreate()

In [2]:
training = spark.read.csv('test1.csv', header=True, inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [3]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [4]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [7]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(
    inputCols=["age", "Experience"],
    outputCol="Independent Features"
)

In [9]:
output = featureassembler.transform(training)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [25]:
finalized_data = output.select("Salary", "Independent Features")
finalized_data.show()

+------+--------------------+
|Salary|Independent Features|
+------+--------------------+
| 30000|         [31.0,10.0]|
| 25000|          [30.0,8.0]|
| 20000|          [29.0,4.0]|
| 20000|          [24.0,3.0]|
| 15000|          [21.0,1.0]|
| 18000|          [23.0,2.0]|
+------+--------------------+



In [31]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol="Independent Features", labelCol="Salary")
regressor = regressor.fit(train_data)

In [32]:
### Coefficients
regressor.coefficients

DenseVector([-1363.6364, 3181.8182])

In [33]:
### Intercepts
regressor.intercept

40454.545454540676

In [34]:
### Prediction
results = regressor.evaluate(test_data)
results.predictions.show()

+------+--------------------+------------------+
|Salary|Independent Features|        prediction|
+------+--------------------+------------------+
| 18000|          [23.0,2.0]|15454.545454545623|
| 20000|          [24.0,3.0]|17272.727272727403|
| 20000|          [29.0,4.0]|13636.363636364687|
+------+--------------------+------------------+

